# MLP Implementation in mlax with Optax loss.
To help with understaning, consider first reading the tutorial on
[Stateful Computations in JAX](https://jax.readthedocs.io/en/latest/jax-101/07-state.html),
especially the Linear Regression worked example. 

You can view the Pytorch reference implementation in `mlp_reference.ipynb`.

This implementation uses Optax loss functions but mlax optimizers.

See `mlp_no_optax.ipynb` for an implementation without using Optax.

See `mlp_optax_optim.ipynb` for an example also using Optax optimizers.

In [1]:
import jax
import jax.numpy as jnp
from jax import (
    nn,
    random,
    tree_util
)
import numpy as np
import optax

We import the `Linear` block from `mlax.nn.blocks` to build the MLP.

We import the `sgd` optimizer to from `mlax.optim` calculate updates on the
model weights, and we import the `apply_gradient` function to apply those
updates.

In [2]:
from mlax.blocks import Linear
from mlax.optim import sgd, apply_updates

We import helpers to load data.

In [3]:
from dataloader import batch, load_mnist

### Load the MNIST dataset as numpy arrays.
We could instead load the datasets as JAX arrays. If we did so, the dataset
will be sent uncommitted to the default device, in my case, the GPU.
Read more about this [here](https://jax.readthedocs.io/en/latest/faq.html#controlling-data-and-computation-placement-on-devices).

Since most datasets are too expensive to copy to to each accelerator, they
should stay on the CPU and be streamed to the accelerators during training and
testing. While we could do this by using `jax.device_put()` on JAX arrays, it's
simpler to keep them as numpy arrays, which always stay on the CPU.

In [4]:
(X_train, y_train), (X_test, y_test) = load_mnist("../data")
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


### Batch the datasets
Data stay as numpy arrays, and therefore on the CPU.

In [5]:
batch_size = 64
X_train, y_train = batch(X_train, y_train, batch_size)
X_test, y_test = batch(X_test, y_test, batch_size)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

938 938
157 157


Note that there are more elegant ways to load in Tensorflow or Pytorch data.

Consider reading Jax's
[Training a Simple Neural Network with tensorflow/datasets Data Loading](https://jax.readthedocs.io/en/latest/notebooks/neural_network_with_tfds_data.html) and
[Training a Simple Neural Network, with PyTorch Data Loading](https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html).

### Initialize MLP model weights.
`model_init` uses a `jax.random.PRNGKey` when initializing the weights.
Read more about random numbers in JAX [here](https://jax.readthedocs.io/en/latest/jax-101/05-random-numbers.html).

The initialized `model_weights` are JAX arrays, so they reside on the default
device, in my case the GPU.

Furthermore, default weights of `mlax` transformations and blocks are of the type
`float32`. You can override that with `init` functions' `dtype` parameter.

In [6]:
def model_init(key):
    key1, key2, key3 = random.split(key, 3)
    
    # Initialize weights for each linear block on the GPU, default type: float32
    w1 = Linear.init(key1, 28*28, 512)
    w2 = Linear.init(key2, 512, 512)
    w3 = Linear.init(key3, 512, 10)
    return [w1, w2, w3]

model_weights = model_init(random.PRNGKey(24))

### Define MLP dataflow

We first explicitly convert its type to `float32`, the type of our
model weights.

We then flatten the 2D numpy array with `jax.numpy.reshape`, converting it
into a JAX array, which gets sent to the default device, in my case the GPU.

Knowing what types we are working with is important because mlax functions do
not promote types implicitly.

We then pass the flattened inputs through several `mlax.nn.Linear` blocks.

Note that the last block does not end with a softmax function, as the Optax loss
function we will use later takes in logits, not probabilities.

In [7]:
@jax.jit
def model_fwd(
    x_batch: np.array,
    weights: jnp.array
):

    # Explicit type promotion
    x_batch = x_batch.astype("float32") 
    
    # Flatten numpy array and send it to GPU.
    n_batches = x_batch.shape[0]
    x_batch = jnp.reshape(x_batch, (n_batches, -1))

    w1, w2, w3 = weights
    x_batch = Linear.fwd(x_batch, w1, nn.relu)
    x_batch = Linear.fwd(x_batch, w2, nn.relu)
    x_batch = Linear.fwd(x_batch, w3, None)
    return x_batch

### Define bach loss function
We use Optax's crossentropy loss.

Note that target values are numpy arrays, which gets streamed from CPU and gets
converted into a JAX array on the GPU implicitly by Optax.

In [8]:
@jax.jit
def loss_fn(
    batched_preds: jnp.array,
    batched_targets: np.array
):
    return optax.softmax_cross_entropy_with_integer_labels(
        batched_preds,
        batched_targets
    ).mean()

We also define a convenience function that calculates the model loss based on
datasets' batched inputs.

In [9]:
def model_loss(
    x_batch: np.array,
    y_batch: np.array,
    weights: jnp.array
):
    return loss_fn(model_fwd(x_batch, weights), y_batch)

### Define optimizer state and function
As with model weights, the optimzer state is a JAX array that gets sent to the
default device.

During initialization, the optimizer infers the type of its state from the
model weights. In our case then, the optimizer state is also of dtype `float32`.

We create an optimizer function that takes in gradients and an optimizer state,
and returns new gradients to be applied and a new optimizer state.

Note we used `jax.tree_util.Partial` instead of `functools.Partial` to allow
using this function in `jit` compiled functions. Read more about this [here](https://jax.readthedocs.io/en/latest/_autosummary/jax.tree_util.Partial.html?highlight=Partial)

In [10]:
# Initialize optimizer state on the GPU
optim_state = sgd.init(model_weights)
optim_fn = tree_util.Partial(sgd.step, lr=5e-4, momentum=0.9)

### Define training step
We first calculate the batch loss, which will only be used for logging.

We then use JAX's `value_and_grad` to calculate the batch loss and the
gradients with repect to `model_weights`. Read more about JAX's autodiff
[here](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#taking-derivatives-with-grad).

The batch loss is only used for logging, but the gradients are passed to
`optim_fn` to get new gradients and a new optimizer state.

We apply the new gradients to update the model weights.

Finally, we return the batch loss, new `model_weights`, and new `optim_state`.

In [11]:
@jax.jit
def train_step(
    x_batch: np.array, 
    y_batch: np.array,
    model_weights: jnp.array,
    optim_fn, # (gradients, optim_state) -> (new_gradients, new_optim_state)
    optim_state: jnp.array
):
    # Find batch loss and gradients
    loss, gradients = jax.value_and_grad(
        model_loss,
        argnums=2 # gradients wrt model_weights (argument 2)
    )(x_batch, y_batch, model_weights)

    # Get new gradients and optimizer state
    gradients, optim_state = optim_fn(gradients, optim_state)

    # Update model_weights with new gradients
    model_weights = apply_updates(gradients, model_weights)
    return loss, model_weights, optim_state

### Define functions for training and testing loops

In [12]:
def train_epoch(X_train, y_train, model_weights, optim_fn, optim_state):
    num_batches = len(X_train)
    train_loss = 0.0
    for i in range(num_batches):
        x_batch, y_batch = X_train[i], y_train[i]
        loss, model_weights, optim_state = train_step(
            x_batch, y_batch,
            model_weights,
            optim_fn, optim_state
        )
        train_loss += loss

    print(f"Train loss: {train_loss / num_batches}") 
    return model_weights, optim_state

In [13]:
def test(X_test, y_test, model_weights):
    num_batches = len(X_test)
    test_loss, accuracy = 0, 0.0
    for i in range(num_batches):
        x_batch, y_batch = X_test[i], y_test[i]
        preds = model_fwd(
            x_batch, model_weights
        )
        loss = loss_fn(preds, y_batch)
        test_loss += loss
        accuracy += (jnp.argmax(preds, axis=1) == y_batch).sum() / len(x_batch)
    
    print(f"Test loss: {test_loss / num_batches}, accuracy: {accuracy / num_batches}")

In [14]:
def train_loop(
    X_train, y_train,
    X_test, y_test,
    model_weights,
    optim_fn, optim_state,
    epochs, test_every
):
    for i in range(epochs):
        epoch = i + 1
        print(f"Epoch {epoch}\n----------------")
        model_weights, optim_state = train_epoch(
            X_train, y_train,
            model_weights,
            optim_fn, optim_state
        )
        if (epoch % test_every == 0):
            test(X_test, y_test, model_weights)
        print(f"----------------")
    
    return model_weights, optim_state

### Train MLP on MNIST dataset

In [15]:
new_model_weights, new_optim_state = train_loop(
    X_train, y_train,
    X_test, y_test,
    model_weights,
    optim_fn, optim_state,
    50, 5
)

Epoch 1
----------------
Train loss: 1.768882393836975
----------------
Epoch 2
----------------
Train loss: 0.15430843830108643
----------------
Epoch 3
----------------
Train loss: 0.09648335725069046
----------------
Epoch 4
----------------
Train loss: 0.06552977859973907
----------------
Epoch 5
----------------
Train loss: 0.04472680017352104
Test loss: 0.15082146227359772, accuracy: 0.9633758068084717
----------------
Epoch 6
----------------
Train loss: 0.031187959015369415
----------------
Epoch 7
----------------
Train loss: 0.02150675654411316
----------------
Epoch 8
----------------
Train loss: 0.015428357757627964
----------------
Epoch 9
----------------
Train loss: 0.010454383678734303
----------------
Epoch 10
----------------
Train loss: 0.007274230010807514
Test loss: 0.1621200293302536, accuracy: 0.9669586420059204
----------------
Epoch 11
----------------
Train loss: 0.005341990850865841
----------------
Epoch 12
----------------
Train loss: 0.004018120933324099
-